In [142]:
import pandas as pd
import openpyxl
import numpy as np
import datetime

## IMPORT CLIMIFY DATA

df = pd.read_csv("skylab_indoor_climate_dec_jan.csv")
# Clean time column by analyzing date from a string and convert to datetime
df['_time'] = pd.to_datetime(df['_time'].str[:10] + ' ' + df['_time'].str[11:19])

df   = df[(df['locationid'] == 229)|(df['locationid'] == 228)|(df['locationid'] == 227)|(df['locationid'] == 224)|(df['locationid'] == 212)]

# Clean Data
#df['locationid'] = df['locationid'].replace(df.loc[df['locationid'].str.contains('229')]['locationid'],'LEC-008-Skylab-3Dprint')
df['locationid'] = df['locationid'].replace(229, 'Print')
df['locationid'] = df['locationid'].replace(228, 'MultiLab')
df['locationid'] = df['locationid'].replace(227, 'DeveloperHall')
df['locationid'] = df['locationid'].replace(224, 'RapidPrototyping')
df['locationid'] = df['locationid'].replace(212, 'SkylabDigital')


# Group the dataframe by the attribute column
grouped = df.groupby('locationid')

# Iterate through each group and create a new dataframe for each group with the group name as the dataframe name
for name, group in grouped:
    globals()[name] = group 

# # Print the dataframes
# print(Print)
# print(MultiLab)
# print(DeveloperHall)
# print(RapidPrototyping)
# print(SkylabDigital)

# Save the dataframes to Excel
# with pd.ExcelWriter('data_climify.xlsx') as writer:
#     print_function.to_excel(writer, sheet_name='Print')
#     MultiLab.to_excel(writer, sheet_name='MultiLab')
#     DeveloperHall.to_excel(writer, sheet_name='DeveloperHall')
#     RapidPrototyping.to_excel(writer, sheet_name='RapidPrototyping')
#     SkylabDigital.to_excel(writer, sheet_name='SkylabDigital')

# Combine the dataframes into one
data_climify = pd.concat([Print, MultiLab, DeveloperHall, RapidPrototyping, SkylabDigital])
# Create a new column that calculates the change in CO2 concentration over a 10-minute period
data_climify['co2_change'] = df['co2'].rolling(window=10).apply(lambda x: x[-1] - x[0], raw=True)

# Create a new column for occupancy
data_climify['occupancy_pred'] = 0

# Calculate occupancy for each room and add it to the dataframe given co2_change and motion values
for i, row in data_climify.iterrows():
    if row['co2_change'] > 0 and row['motion'] > 3:
        data_climify.loc[i, 'occupancy_pred'] = 1  
    else:
        data_climify.loc[i, 'occupancy_pred'] = 0  

 
# data_climify.drop('_time', axis=1, inplace=True)

# # Export data_climify to Excel
data_climify.to_excel('data_climify.xlsx')

# preview first 5 rows of data_climify
data_climify.head()


In [144]:
## IMPORT OCCUPANCY FROM SKYLAB

df = pd.read_excel("DTU SkyLab Usage History Jan 2022_Jan 2023_NoNAME.xlsx")

# Clean the data and keep only the values for the Resource we are interested in
df = df[(df['Resource'] == 'LEC-008-Skylab-3Dprint')|(df['Resource'] == 'LEC-010-Skylab-Multi-Lab')|(df['Resource'] == 'LEC-008-Skylab-DeveloperHall')|(df['Resource'] == 'LEC-008-Skylab-RapidPrototyping')|(df['Resource'] == 'LEC-012-Skylab Digi')]
# Rename the Resource attributes to match the names in the climify data
df['Resource'] = df['Resource'].replace('LEC-008-Skylab-3Dprint', 'Print')
df['Resource'] = df['Resource'].replace('LEC-010-Skylab-Multi-Lab', 'MultiLab')
# df['Resource'] = df['Resource'].replace('LEC-008-Skylab-DeveloperHall', 'DeveloperHall')
# df['Resource'] = df['Resource'].replace('LEC-008-Skylab-RapidPrototyping', 'RapidPrototyping')
df['Resource'] = df['Resource'].replace('LEC-012-Skylab Digi', 'SkylabDigital')

# Convert the start and end dates to datetime
df['Usage Start'] = pd.to_datetime(df['Usage Start'])
df['Usage End'] = pd.to_datetime(df['Usage End'])

# Divide the data into groups based on the Resource attribute
grouped = df.groupby('Resource')

# Iterate through each group and create a new dataframe for each group with the group name as the dataframe name amd with dates as the index and occupancy as the column with 1s for the dates within the start and end date range
for name, group in grouped:
    globals()[name] = pd.DataFrame(index=pd.date_range(group['Usage Start'].min(), group['Usage End'].max()))
    for i, row in group.iterrows():
        globals()[name].loc[row['Usage Start']:row['Usage End'], 'occupancy'] = 1
# Add 0s for the dates outside the start and end date range for each dataframe
for name, group in grouped:
    globals()[name].fillna(0, inplace=True)
    
# Add a column in each dataframe with the corresponding name of the dataframe
for name, group in grouped:
    globals()[name]['locationid'] = name
# Combine the dataframes into one
data_skylab = pd.concat([Print, MultiLab, DeveloperHall, RapidPrototyping, SkylabDigital])

# Convert the date in index to a columm
data_skylab.reset_index(inplace=True)
data_skylab.rename(columns={'index': '_time'}, inplace=True)


# Create a new column for occupancy
# data_climify['occupancy'] = 0

# # Add the occupancy values from the skylab data to the climify data
# for i, row in data_climify.iterrows():
#     for j, row2 in data_skylab.iterrows():
#         if row['locationid'] == row2['locationid'] and row['_time'] == row2['seconds']:
#             data_climify.loc[i, 'occupancy'] = row2['occupancy']
# # Print the dataframe
# print(data_climify)

# # Create a new DataFrame with the dates as the index
# data_timesteps = pd.DataFrame(index=pd.date_range(df['Usage Start'].min(), df['Usage End'].max()))

# # Add a column with 1s for the dates within the start and end date range
# for i, row in df.iterrows():
#     data_timesteps.loc[row['Usage Start']:row['Usage End'], 'occupancy'] = 1

# # Fill any missing values with 0s
# data_timesteps.fillna(0, inplace=True)

# # Include the corresponding Resource attribute for each date and in between the dates time range
# for i, row in df.iterrows():
#     data_timesteps.loc[row['Usage Start']:row['Usage End'], 'locationid'] = row['Resource']

# print(data_timesteps)

In [129]:
# Import streamlit
import streamlit as st
import datetime

# Create a streamlit app to select a date range and display the occupancy for each room
# Create a sidebar with a date range selector
st.sidebar.header('Select a date range')
start_date = st.sidebar.date_input('Start date', datetime.date(2021, 1, 1))
end_date = st.sidebar.date_input('End date', datetime.date(2021, 1, 31))

In [131]:
import streamlit as st
import pandas as pd

data = data_climify
# Convert the date column to datetime
data['_time'] = pd.to_datetime(data['_time'])

# Create a function to filter the data by date range
def filter_data(start_date, end_date):
    filtered_data = data[(data['_time'] >= start_date) & (data['_time'] <= end_date)]
    return filtered_data

# Create the Streamlit app
st.title('Data Viewer')

# Add a date picker for the start date

start_date = st.date_input('Select a start date')

# Add a date picker for the end date
end_date = st.date_input('Select an end date')

# Add a button to filter the data
if st.button('Filter'):
    filtered_data = filter_data(start_date, end_date)
    st.dataframe(filtered_data)

In [134]:
# Load the data
data = pd.read_excel('data_climify.xlsx')
# Print the first 5 rows of the data
data.head()

,Unnamed: 0,locationid,co2,humidity,light,motion,temperature,co2_change,occupancy_pred,seconds
0,109801,Print,406,36,3,0,21.3,-20.0,0,2022-12-26 14:11:52
1,109802,Print,411,36,3,0,21.3,-15.0,0,2022-12-26 14:16:52
2,109803,Print,410,36,3,0,21.4,-32.0,0,2022-12-26 14:21:52
3,109804,Print,418,36,3,0,21.3,-21.0,0,2022-12-26 14:26:52
4,109805,Print,409,36,3,0,21.3,-26.0,0,2022-12-26 14:31:52
